# **SVD Model**

## **Installing Surprise**

In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 6.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675382 sha256=1ce9daf846d5cb5517cb29c7d484a50aa024e09923d01330328d7f08b0fe7f01
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


## **Importing libraries**

In [0]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split

## **Preparing data**

In [0]:
ratings = pd.read_csv('ratings.csv')
books = pd.read_csv('books.csv')
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['book_id', 'user_id', 'rating']], reader)

## **Training**

In [0]:
train_set, test_set = train_test_split(data, test_size=0.2)
model = SVD(n_epochs=20, lr_all=0.005, reg_all=0.2)
model.fit(train_set)
predictions = model.test(test_set)

## **Preparing the dataset with predictions**

In [5]:
pred_ds = pd.DataFrame(predictions, columns=['book_id', 'user_id', 
                                             'actual_rating', 'pred_rating', 
                                             'details'])
pred_ds['impossible'] = pred_ds['details'].apply(lambda x: x['was_impossible'])
pred_ds['pred_rating_round'] = pred_ds['pred_rating'].round()
pred_ds['abs_err'] = abs(pred_ds['pred_rating'] - pred_ds['actual_rating'])
pred_ds.drop(['details'], axis=1, inplace=True)
pred_ds

,book_id,user_id,actual_rating,pred_rating,impossible,pred_rating_round,abs_err
0,4080,7127,4.0,3.700083,False,4.0,0.299917
1,8681,15706,5.0,4.019954,False,4.0,0.980046
2,3971,50940,5.0,4.235722,False,4.0,0.764278
3,5696,51794,2.0,3.842757,False,4.0,1.842757
4,2435,7942,4.0,4.035741,False,4.0,0.035741
...,...,...,...,...,...,...,...
196347,7016,21252,5.0,4.560632,False,5.0,0.439368
196348,9306,41615,3.0,3.589586,False,4.0,0.589586
196349,2443,49180,5.0,3.944374,False,4.0,1.055626
196350,8224,23823,3.0,3.503658,False,4.0,0.503658


## **Viewing the results**

In [7]:
ratings_titles = ratings.merge(books[['book_id', 'title']], on='book_id', 
                               how='left')
ratings_titles = ratings_titles.merge(pred_ds[['book_id', 'user_id', 
                                               'pred_rating']], 
                                      on=['book_id', 'user_id'], how='left')
id_user = 588
user_ds = ratings_titles[ratings_titles['user_id'] == id_user]
res_ds = user_ds[user_ds['pred_rating'].notna()].sort_values('rating', 
                                                             ascending=False)

res_ds


,book_id,user_id,rating,title,pred_rating
11401,115,588,5,NaN,3.707578
32206,323,588,4,NaN,3.637411
31504,316,588,4,NaN,3.916544
528259,5299,588,4,NaN,3.589206
173291,1734,588,4,NaN,4.090564
100809,1009,588,4,NaN,3.478527
66103,662,588,4,Atlas Shrugged,3.991486
58602,587,588,4,NaN,3.633395
53702,538,588,4,NaN,3.561054
51505,516,588,4,NaN,3.761011


In [12]:
bid = [662, 11, 24, 33, 231]
for b in bid:
  print(ratings.query(f'book_id == {b} and user_id == 588'), '\n')

       book_id  user_id  rating
66099      662      588       4 

      book_id  user_id  rating
1001       11      588       4 

      book_id  user_id  rating
2300       24      588       4 

      book_id  user_id  rating
3201       33      588       3 

       book_id  user_id  rating
23000      231      588       2 

